In [3]:
from itertools import zip_longest
from collections import Counter
import numpy as np

def hamming_distance(str1, str2):
  return sum (c1 != c2 for c1, c2 in zip_longest(str1, str2))

#converts to binary (either base pair = '1') to calculate paired vs unpaired distance
def convert(string):
  string = string.replace("(", "1")
  string = string.replace(")", "1")
  string = string.replace(".", "0")
  return string

#this function doesn't work for pseudoknotted structures, only knot-free
def get_basepairs(string, start=0):     
    bp_stack = []
    bp_output = []
    for i in range(len(string)):
        if string[i] == '.':
            next
        if string[i] == '(':
            bp_stack.append(i)
        if string[i] == ')':
            bp_output.append((bp_stack.pop()+start+1, i+start+1))  #this right here should maybe be +start+1, i+start-1
    return bp_output

#find loops/bulges of structure (...) where any number of . is possible but it must be consistent
def get_loops(string, start=0):
    loop_stack = []
    loop_output = []
    loop_open = False
    bulge_open = False
    for i in range(len(string)):
        if string[i] == '(':
            if string[i+1] == '.':
                loop_open = True
        if string[i] == ')':
            if i == len(string)-1:
                break
            if string[i+1] == '.':
                bulge_open = True
        if loop_open and string[i] == '.':
            loop_stack.append(i+start+1)
            next
        if bulge_open and string[i] == '.':
            loop_stack.append(i+start+1)
            next
        if loop_open and string[i] == ')' or loop_open and string[i] == '(':
            if string[i-1] == '.':
                loop_open = False
                loop_stack = sorted(loop_stack)
                loop_output.append(tuple(loop_stack))
                loop_stack = []
        if bulge_open and string[i] == ')':
            if string[i-1] == '.':
                bulge_open = False
                loop_stack = sorted(loop_stack)
                loop_output.append(tuple(loop_stack))
                loop_stack = []
        if bulge_open and string[i] == '(':
            bulge_open = False
            loop_stack = []
    return loop_output
        
def compare(list_of_strings, start=0):
    list_of_hits = []
    number_of_lists = len(list_of_strings)
    for item in list_of_strings:
        list_of_hits.append(get_loops(item, start))
        #list_of_hits.append(get_basepairs(item, start))
    list_of_hits = [item for sublist in list_of_hits for item in sublist]
    tally_of_hits = Counter(list_of_hits)
    #common_bp gets the list of tuple base pair positions that exist in all of the structures compared
    common_bp = [key for key in tally_of_hits.keys() if tally_of_hits[key] == number_of_lists]
    return common_bp

def levenshtein_distance(str1, str2):
    matrix = np.zeros((len(str1)+1, len(str2)+1))
    for x in range(len(str1)+1):
        matrix[x,0] = x
    for y in range(len(str2)+1):
        matrix[0,y] = y
    for x in range(1, len(str1)+1):
        for y in range(1, len(str2)+1):
            if str1[x-1] == str2[y-1]:
                matrix[x,y] = min(
                    matrix[x-1, y]+1,
                    matrix[x-1, y-1],
                    matrix[x, y-1]+1
                )
            else: 
                matrix[x,y] = min(
                    matrix[x-1, y]+1,
                    matrix[x-1, y-1]+1,
                    matrix[x, y-1]+1
                )
    return matrix[len(str1), len(str2)]

In [3]:
R = '..............(.(((((((((..(((((.((....)).)))))..)))))......(((((((((((((((((((......)))))))).)))))))..)))).....(((((((........)))))))...)))).)....'
L = '............((........))......((((......))))...(..((((((.(((((........(((((.((((((......)))))).))))))))))))))))..).......((((.(..(((((.((....)).)))))..)))))......'
C = '...................((((((...(((((.((((................................(((((((.((((....)))).)))))))((((......))))....)))).)))))....)))))).(((((((......)))))))...'

print(hamming_distance(convert(R), convert(L)), hamming_distance(convert(R), convert(C)), hamming_distance(convert(L), convert(C)))

88 78 66


In [229]:
R = '..............(.(((((((((..(((((.((....)).)))))..)))))......(((((((((((((((((((......)))))))).)))))))..)))).....(((((((........)))))))...)))).)....'
L = '............((........))......((((......))))...(..((((((.(((((........(((((.((((((......)))))).))))))))))))))))..).......((((.(..(((((.((....)).)))))..)))))......'
C = '...................((((((...(((((.((((................................(((((((.((((....)))).)))))))((((......))))....)))).)))))....)))))).(((((((......)))))))...'
S = '..........((((................((((......))))......((((((.(((((......(((((.((((((......)))))).))))))))))))))))..))))..(((((..(((((.((....)).)))))..)))))......'
print(compare([R, L, C, S]))

[]


In [58]:
R = '..............(.(((((((((..(((((.((....)).)))))..)))))......(((((((((((((((((((......)))))))).)))))))..)))).....(((((((........)))))))...)))).)....'
L = '............((........))......((((......))))...(..((((((.(((((........(((((.((((((......)))))).))))))))))))))))..).......((((.(..(((((.((....)).)))))..)))))......'
C = '...................((((((...(((((.((((................................(((((((.((((....)))).)))))))((((......))))....)))).)))))....)))))).(((((((......)))))))...'
print(levenshtein_distance(R, L), levenshtein_distance(R,C), levenshtein_distance(L, C))

57.0 64.0 71.0


In [7]:
print(hamming_distance(R,L), hamming_distance(convert(R), convert(L)), levenshtein_distance(convert(R), convert(L)))

85 71 54.0


In [1]:
import RNA


ModuleNotFoundError: No module named 'RNA'

In [136]:
RNA.energy_of_structure('UYnCnUUAAUUGACUUCUAUUUGUGCUUUUUAGCCUUUCUGCUAUUCCUUGUUUUAAUnAUGCUUAUUAUnUUUUGGUUYUCACUYGAAnUnCARGAUCnnRAnGAAnCUUGUnnC---GYCUAAACGAACAUGAAAYUUCUYRUUnUYUURRnnnnYnU', 
                        '...................((((((...(((((.((((................................(((((((.((((....)))).)))))))((((......))))....)))).)))))....)))))).(((((((......)))))))...',
                        0)

17.899999618530273

In [45]:
RNA.fold("UUUCAUUAAUUGACUUCUAUUUGUGCUUUUUAGCCUUUCUGCUAUUCCUUGUUUUAAUUAUGCUUAUUAUCUUUUGGUUCUCACUUGAACUGCAAGAUCAUAAUGAAACUUGUCACGCCUAAACGAACAUGAAAUUUCUUGUUUUCUUAGGAAUCAU")

['..........((((................((((......))))......((((((.(((((......(((((.((((((......)))))).))))))))))))))))..))))..(((((..(((((.((....)).)))))..)))))......',
 -27.200000762939453]

In [121]:
string = '...((..))...((...((...))))...'
print(get_loops(string))

[[6, 7], [15, 16, 17], [20, 21, 22]]


In [176]:
SHAPE = '..((.......))...........(((....)))....))))))........................(((((.((((((......)))))).)))))...................(((((..(((((.((....)).)))))..)))))......' 
RNAz =  '..........((((................((((......))))......((((((.(((((......(((((.((((((......)))))).))))))))))))))))..))))..(((((..(((((.((....)).)))))..)))))......'
print(levenshtein_distance(SHAPE,RNAz))

41.0


In [54]:
Zie = '..........((((................((((......))))......((((((.(((((......(((((.((((((......)))))).))))))))))))))))..))))..(((((..(((((.((....)).)))))..)))))......'
Li  = '..))))...))..)))((((.(((((.....)))))..))))..........((.((((((.....)))))).)).....(((((.((((((.((((((.((((.((.....((((((((...((((.(((((.((((((...))))))))))).))'
print(levenshtein_distance(Zie,Li))

82.0


In [95]:
And =   '.....................................................................................................................(((((..(((((.((....)).)))))..)))))......'
Hus =   '..(((.....))).))).))).(((...((((((......))))...........))...)))((((.(((((.((((((......)))))).))))).)))).((((((.((....(((((..(((((.((....)).)))))..)))))......'
Mvivo = '..((.......))...........(((....)))....))))))........................(((((.((((((......)))))).)))))...................(((((..(((((.((....)).)))))..)))))......'
Li =    '..))))...))..)))((((.(((((.....)))))..))))..........((.((((((.....)))))).)).....(((((.((((((.((((((.((((.((.....((((((((...((((.(((((.((((((...))))))))))).))'
print(levenshtein_distance(And,Hus), levenshtein_distance(And, Mvivo), levenshtein_distance(And, Li), levenshtein_distance(Hus,Mvivo), levenshtein_distance(Hus, Li), levenshtein_distance(Mvivo, Li))

68.0 38.0 89.0 39.0 74.0 77.0


In [8]:
sarbec = '......(((((((.(((....)))))..)))))...........(((((.....))))).((((.......)).))..............((((((((.((.((((.(((.....))).)))))).))))))))........................((((((((((((.(((((...(((.(((.(((((((..((((((.(((((......)))))..))))))......)))(((((((.((......)))))))))(((....))))))).)))))).))))))))))...))))))).....'
sarbec_window = sarbec[80:238]
rnaz_0 = '.(((((.(((((((....((((((((((((.....))).....(((((.((((((.....))))))((((.(..((.((...(((((...))))))).))..).))))))))).(((((......))))).)))))))))....)))).))).)))))'
print(levenshtein_distance(convert(sarbec_window),convert(rnaz_0)))
print(convert(sarbec_window))
print(convert(rnaz_0))
print(sarbec_window)

60.0
00000000001111111101101111011100000111011111101111111100000000000000000000000011111111111101111100011101110111111100111111011111000000111110011111100000011111
01111101111111000011111111111100000111000001111101111110000011111111110100110110001111100011111110110010111111111011111000000111110111111111000011110111011111
..........((((((((.((.((((.(((.....))).)))))).))))))))........................((((((((((((.(((((...(((.(((.(((((((..((((((.(((((......)))))..))))))......)))((


In [223]:
sarbec_comp = '1110111011111110011111101111100000011111001111110000001111111111011000000111111111111000011111110111111011111111110001111111000000000000000000000000000000000000'
rnaz_2_comp = '0000001101111110111110000001111100111111000111100111111101100000011111111111100001110011110011111000000001111101111110001100001100001110111011111100011111100110'
print(levenshtein_distance(sarbec_comp,rnaz_2_comp))

54.0


In [21]:
ref = '......(((((((.......))).))))...(((((......)))))..............(((((((((...((((..(((((.((..(((((......))))).)).)))))...(((((....)))))........)))))))))))))........'
mut = '......(((.(((((((((((((.(((...))).)))......)).)))))))).)))...(((((((((...((((..(((((.((..(((((......))))).)).)))))...(((((....)))))........)))))))))))))........'
print(levenshtein_distance(ref,mut))

31.0
